## Imports and Setup

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
import pandas as pd
import re
import seaborn as sns
import string

In [ ]:
from collections import Counter
from pathlib import Path
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud

In [ ]:
from src.paths import INTERIM_CORPUS

In [ ]:
%config InlineBackend.figure_format = "retina"
%matplotlib inline

## Ingestion and Initial Inspection of the Data

### Number of files to collect

In [ ]:
Counter(p.suffix for p in INTERIM_CORPUS.glob('*.csv'))

### Import

In [ ]:
df = pd.concat([
    pd.read_csv(
        f, 
        encoding='utf-8',
        parse_dates=['parsed_created_at'])
    for f in INTERIM_CORPUS.glob('*.csv')
], ignore_index=True)  # otherwise, index is weird

### Data Inspection

In [ ]:
# first five rows
df.head()

In [ ]:
# last five rows
df.tail()

**Variables/Column names**

In [ ]:
df.columns

**Variable/Column types**

In [ ]:
df.dtypes

## Basic Stats

In [ ]:
len(df.index)

### Tweet Types

In [ ]:
types = df.tweet_type.value_counts()
print(types)

In [ ]:
_ = sns.barplot(x=types.index, y=types.values, alpha=0.8)

### Tweet Types by Day

In [ ]:
df.set_index('parsed_created_at', inplace=True)

In [ ]:
df['day'] = df.index.date

In [ ]:
daily_plot = df.groupby(['day', 'tweet_type']).size().unstack()

In [ ]:
_ = daily_plot.plot(title='Daily Tweets by Type', logy=True)

### Users

In [ ]:
# number of unique tweeters
df['user_screen_name'].nunique()

In [ ]:
tweeps = (df.groupby(['user_screen_name'])
         .count()
         .reset_index())

In [ ]:
tweeps = (tweeps
         .sort_values(by='text')
         .tail(10))

In [ ]:
tweeps

In [ ]:
x = tweeps['user_screen_name']
y = tweeps['text']
plt.xlabel('Twitter handle')
plt.ylabel('Number of Tweets')
plt.title('Most number of Tweets by user')
h=plt.bar(range(10), y, label='Most Tweets by user')
xticks_pos = [0.65*patch.get_width() + patch.get_xy()[0] for patch in h]
_ = plt.xticks(xticks_pos, x, ha='right', rotation=45)

### Hashtags

We'll get the hashtags from the text to make sure we get all of them, as the `hashtag` field isn't always reliable. 

In [ ]:
hashtags = (df
            .text
            .str.extractall(r'(\#\w+)')[0]
            .str.lower()
            .value_counts()
           )


In [ ]:
hashtags.nunique()

In [ ]:
hashtags = hashtags[1:11,]

In [ ]:
hashtags

In [ ]:
_ = sns.barplot(x=hashtags.values, y=hashtags.index, alpha=0.8)

### Mentions

In [ ]:
mentions = (df
            .text
            .str.extractall(r'(@[\w_]+)')[0]
            .str.lower()
            .value_counts()
)

In [ ]:
mentions = mentions[:10,]

In [ ]:
_ = sns.barplot(x=mentions.values, y=mentions.index, alpha=0.8)

### Wordcloud

I have my doubts about how informative wordclouds actually are, but maybe they work as art.

In [ ]:
junk = re.compile("al|RT|\n|&.*?;|http[s](?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)*")
tweets = [junk.sub(" ", t) for t in df.text]

vec = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=.5)
tfv = vec.fit_transform(tweets)

terms = vec.get_feature_names()
wc = WordCloud(height=1000, width=1000, max_words=1000).generate(" ".join(terms))

plt.figure(figsize=(10, 10))
plt.imshow(wc)
plt.axis("off")
plt.show()

That included some non-English tweets, so let's correct that. 

In [ ]:
english_df = df[df.lang == 'en']

In [ ]:
junk = re.compile("al|RT|\n|&.*?;|http[s](?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+)*")
tweets = [junk.sub(" ", t) for t in english_df.text]

vec = TfidfVectorizer(stop_words='english', ngram_range=(1,2), max_df=.5)
tfv = vec.fit_transform(tweets)

terms = vec.get_feature_names()
wc = WordCloud(height=1000, width=1000, max_words=1000).generate(" ".join(terms))

plt.figure(figsize=(10, 10))
plt.imshow(wc)
plt.axis("off")
plt.show()